# PRM-02 Tutorial Session

### Author: Jay Parmar

##### Created on: 19th Sep 2020

##### Last Updated on: 15th Sep 2021

### Lecture Agenda
- Portfolio Creation
- Calculating Risk and Returns
- Modern Portfolio Theory
- Security Selection
- Kelly Criterion for Position Sizing
- Calculating Profitability Ratios using Pyfolio

### What this lecture is not?
1. This lecture is *NOT* about Python, but rather Portfolio concepts with the help of Python.

In [ ]:
# This is to ignore the warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyfolio as pf
import cufflinks

%matplotlib inline

pd.options.display.precision = 4
plt.style.use('seaborn-ticks')

In [ ]:
# To display multiple outputs from the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
!pip install pyfolio==0.9.2
!pip install pandas==1.2.1

# 1) Investing in a Single Stock

In [ ]:
# Define the first stock
stk_1 = 'TCS' # IT Company

In [ ]:
# Read stock 1 data and verify its content
stk_1_data = pd.read_csv(stk_1+'.csv', index_col=0, parse_dates=True)

In [ ]:
stk_1_data.head()

In [ ]:
stk_1_data.tail()

### Daily Returns

In [ ]:
# Calculate daily percentage changes for stock 1
stk_1_data['daily_returns'] = stk_1_data['Adj Close'].pct_change()

stk_1_data.dropna(inplace=True)

In [ ]:
# Function to compute net cumulative returns
def compute_cum_returns(data):
    cum_returns = (1 + data).cumprod() - 1
    return cum_returns

In [ ]:
# Calculate cumulative returns for stock 1
stk_1_cum_returns = compute_cum_returns(stk_1_data['daily_returns'])

# Display cumulative returns for stock 1
stk_1_cum_returns.iplot(title='TCS Cumulative Returns', xTitle='Dates', yTitle='Returns')

# Print the final value
print('The cumulative returns for %s are %.3f%%' % (stk_1, stk_1_cum_returns[-1] * 100))

### Annualized Returns

An annualized total return is the geometric average amount of money earned by an investment each year over a given time period. The annualized return formula is calculated as a geometric average to show what an investor would earn over a period of time if the annual return was compounded.

$$ Annualized\ Returns = (1 + Cumulative\ Return)^\frac{365}{N} - 1 $$

Where:

$ 252 = $ Number of trading days in year

$ N = $ Number of trading days for a strategy

In [ ]:
# Function to compute annualized returns
def compute_ann_returns(cum_returns, n):
    trading_days = 252
    
    ann_returns = ((1 + cum_returns) ** (trading_days/n)) - 1
    
    return ann_returns

In [ ]:
# Define length of data
n = len(stk_1_data)

# Compute annualized returns for stk_1
stk_1_annualized_returns = compute_ann_returns(stk_1_cum_returns[-1], n)

print('The annualized returns for %s are %.3f%%' % (stk_1, stk_1_annualized_returns * 100))

##### Questions to ask:-

1. Should we invest in only single stock? If yes, why? If no, why not?
2. Is investing in a single stock risky?

### Daily Volatility

In [ ]:
# Calculate daily volatility for stock 1
stk_1_daily_volatility = stk_1_data.daily_returns.std()

print('The daily volatility of %s is %.3f%%' % (stk_1, stk_1_daily_volatility))

### Annualized Volatility

In [ ]:
# Calculate the annualised volatility for stock 1
stk_1_annual_volatility = stk_1_daily_volatility * np.sqrt(252)

print('The annualised volatility of %s is %.3f%%' % (stk_1, stk_1_annual_volatility * 100))

In [ ]:
# Comparison
comparison = pd.DataFrame(columns=['ann_returns', 'ann_volatility'])

comparison.loc[stk_1, 'ann_returns'] = round(stk_1_annualized_returns * 100, 3)
comparison.loc[stk_1, 'ann_volatility'] = round(stk_1_annual_volatility * 100, 3)

comparison.head()

### Goal: Create a portfolio with annual returns similar to TCS and with lower annual volatility.

# 2) Modern Portfolio Theory

In [ ]:
# Define stock 2
stk_2 = 'MARUTI' # Automobile company

In [ ]:
# Read stock 2 data 
stk_2_data = pd.read_csv(stk_2+'.csv', index_col=0, parse_dates=True)

stk_2_data.head()

In [ ]:
stk_2_data.tail()

In [ ]:
# Compute daily percentage returns for stock 2
stk_2_data['daily_returns'] = stk_2_data['Adj Close'].pct_change()

stk_2_data.dropna(inplace=True)

In [ ]:
# Calculate cumulative returns for stock 2
stk_2_cum_returns = compute_cum_returns(stk_2_data['daily_returns'])

# Display cumulative returns for stock 2
stk_2_cum_returns.iplot(title='Maruti Cumulative Returns', xTitle='Dates', yTitle='Returns')

# Print the final value
print('The cumulative returns for %s are %.3f%%' % (stk_2, stk_2_cum_returns[-1] * 100))

In [ ]:
# Number of trading days for stock 2
n = len(stk_2_data)

# Compute annualized returns for stock 2
stk_2_annualized_returns = compute_ann_returns(stk_2_cum_returns[-1], n)

print('The annualized returns for %s are %.3f%%' % (stk_2, stk_2_annualized_returns * 100))

In [ ]:
# Calculate daily volatility for stock 2
stk_2_daily_volatility = stk_2_data.daily_returns.std()

print('The daily volatility of %s is %.3f%%' % (stk_2, stk_2_daily_volatility))

In [ ]:
# Calculate the annualised volatility for stock 2
stk_2_annual_volatility = stk_2_daily_volatility * np.sqrt(252)

print('The annualised volatility of %s is %.3f%%' % (stk_2, stk_2_annual_volatility * 100))

In [ ]:
# Update stock 2 data in 'comparison' dataframe
comparison.loc[stk_2, 'ann_returns'] = round(stk_2_annualized_returns * 100, 3)
comparison.loc[stk_2, 'ann_volatility'] = round(stk_2_annual_volatility * 100, 3)

comparison.head()

#####  Questions to ask:

1. Can we do anything to aim similar returns as stock 2, but with lower risk?

In [ ]:
# Defining weights for stock 1 and 2 - Equal Allocations
weight_in_stk_1 = 0.5 # This is 50%
weight_in_stk_2 = 1 - weight_in_stk_1

### a) Returns of a portfolio

$$ Portfolio\ Returns = (w_A * R_A) + (w_B * R_B)$$

Where:

A = Stock A
<br>B = Stock A

$ w_A = $ Weight in stock A

$ R_A = $ Returns of stock A

$ w_B = $ Weight in stock B

$ R_B = $ Returns of stock B


$$ Portfolio\ Returns = (w_A * R_A) + (w_B * R_B) + (w_C * R_C) + (w_D * R_D)$$

In [ ]:
# Calculate portfolio returns
port_daily_ret = (weight_in_stk_1 * stk_1_data['daily_returns']) + (weight_in_stk_2 * stk_2_data['daily_returns'])

# Storing in a new variable which we will use later
stk_1_2_daily_returns = port_daily_ret

In [ ]:
# Calculate cumulative portfolio returns
port_cum_returns = compute_cum_returns(port_daily_ret)

# Print the final value
print('The cumulative returns of a portfolio with %s and %s are %.3f%%' % (stk_1, stk_2, port_cum_returns[-1] * 100))

In [ ]:
# Calculate annualized portfolio returns
port_annualized_returns = compute_ann_returns(port_cum_returns[-1], len(port_cum_returns))

print('The annualized returns of a portfolio with %s and %s are %.3f%%' % (stk_1, stk_2, port_annualized_returns * 100))

##### Questions to ask:

- How would you compute returns of a portfolio with more than two stocks?

### b) Volatility of a Portfolio

$$ \text{Portfolio Variance} = w^2_A \times \sigma^2(R_A) + w^2_B \times \sigma^2(R_B) + 2 \times w_A \times w_B \times cov(R_A, R_B) $$

$$ \text{Portfolio Std Dev} = \sqrt{w^2_A \times \sigma^2(R_A) + w^2_B \times \sigma^2(R_B) + 2 \times w_A \times w_B \times cov(R_A, R_B)} $$

Where:-

$ w_A = $ Weight in stock A

$ \sigma^2(R_A) = $ Variance of returns of stock A

$ w_B = $ Weight in stock B

$ \sigma^2(R_B) = $ Variance of returns of stock B

$ cov(R_A, R_B) = $ Corvariance between stock A returns and stock B returns

In [ ]:
# Calculate covariance between stock 1 and 2
stk_1_2_cov = np.cov(stk_1_data['daily_returns'], stk_2_data['daily_returns'])

In [ ]:
stk_1_2_cov

In [ ]:
var_cov_matrix = pd.DataFrame(stk_1_2_cov, columns=['TCS', 'Maruti'], index=['TCS', 'Maruti'])

var_cov_matrix

###### Covariance of X with itself is a variance of X.

- Covariance of TCS with TCS is the variance of TCS. (which can also be calculated using `np.var`)

In [ ]:
print('The covariance between %s and %s is %.6f' % (stk_1, stk_2, stk_1_2_cov[0, 1]))

In [ ]:
# Calculate daily volatility of a portfolio
port_daily_variance = (weight_in_stk_1 * stk_1_daily_volatility) ** 2 + (weight_in_stk_2 * stk_2_daily_volatility) ** 2 + \
                        2 * weight_in_stk_1 * weight_in_stk_2 * stk_1_2_cov[0, 1]

port_daily_volatility = np.sqrt(port_daily_variance)

print('The daily volatility of a portfolio with %s and %s is %.3f%%' % (stk_1, stk_2, port_daily_volatility))

In [ ]:
# Calculate the annualised volatility of the portfolio
port_annualized_volatility = port_daily_volatility * np.sqrt(252)

print('The annualised volatility of a portfolio with %s and %s is %.3f%%' % (stk_1, stk_2, port_annualized_volatility * 100))

In [ ]:
# Update the portfolio data in 'comparison' dataframe
comparison.loc[stk_1 + '_' + stk_2, 'ann_returns'] = round(port_annualized_returns * 100)
comparison.loc[stk_1 + '_' + stk_2, 'ann_volatility'] = round(port_annualized_volatility * 100, 3)

comparison.head()

##### Questions to ask:

1. Did investing in two stocks reduced the risk?
2. What can be the potential problem with the current portfolio?
3. Can we further reduce the risk?

In [ ]:
# Correlation between stock 1 and 2
corr = np.corrcoef(stk_1_data['daily_returns'], stk_2_data['daily_returns'])[0, 1]

print('A correlation between %s and %s is %.3f' % (stk_1, stk_2, corr))

# 3) Investing in Two Uncorrelated Securities

### a) Download Stock Data and Compute Volatility and Returns

In [ ]:
# Define stock 3
stk_3 = 'MARICO' # FMCG Company

In [ ]:
# Read stock 3 data
stk_3_data = pd.read_csv(stk_3 + '.csv', index_col=0, parse_dates=True)

# Verify the data
stk_3_data.head()

In [ ]:
# Compute daily percentage returns for stock 3
stk_3_data['daily_returns'] = stk_3_data['Adj Close'].pct_change()

stk_3_data.dropna(inplace=True)

In [ ]:
# Calculate cumulative returns for stock 3
stk_3_cum_returns = compute_cum_returns(stk_3_data['daily_returns'])

# Display cumulative returns for stock 3
stk_3_cum_returns.iplot(title='Marico Cumulative Returns', xTitle='Dates', yTitle='Returns')

# Print the final value
print('The cumulative returns of %s are %.3f%%' % (stk_3, stk_3_cum_returns[-1] * 100))

In [ ]:
# Calculate daily volatility for stock 3
stk_3_daily_volatility = stk_3_data.daily_returns.std()

print('The daily volatility of %s is %.3f%%' % (stk_3, stk_3_daily_volatility))

### b) Check Correlation

In [ ]:
# Check correlation between stock 1 and stock 3
corr = np.corrcoef(stk_1_data['daily_returns'], stk_3_data['daily_returns'])[0, 1]

print('A correlation between %s and %s is %.3f' % (stk_1, stk_3, corr))

### c) Build Portfolio and Calculate Expected Returns

In [ ]:
# Defining weights for stock 1 and stock 3 - Equal Allocations
weight_in_stk_1 = 0.5
weight_in_stk_3 = 1 - weight_in_stk_1

In [ ]:
# Calculate portfolio returns
port_daily_returns = (weight_in_stk_1 * stk_1_data['daily_returns']) + (weight_in_stk_3 * stk_3_data['daily_returns'])

# Storing in a new variable which we will use later
stk_1_3_daily_returns = port_daily_returns

In [ ]:
# Calculate cumulative portfolio returns
port_cum_returns = compute_cum_returns(port_daily_returns)

# Print the final value
print('The cumulative return of a portfolio with %s and %s is %.3f%%' % (stk_1, stk_3, port_cum_returns[-1] * 100))

In [ ]:
# Calculate annualized returns of the portfolio
port_annualized_returns = compute_ann_returns(port_cum_returns[-1], len(port_cum_returns))

print('The annualized returns of a portfolio with %s and %s are %.3f%%' % (stk_1, stk_3, port_annualized_returns * 100))

### d) Calculate Standard Deviation of a Portfolio

In [ ]:
# Calculate the covariance between stock 1 and stock 3
stk_1_3_cov = np.cov(stk_1_data['daily_returns'], stk_3_data['daily_returns'])

print('The covariance between %s and %s is %.6f' % (stk_1, stk_3, stk_1_3_cov[0, 1]))

In [ ]:
# Calculate daily volatility of a portfolio
port_daily_variance = (weight_in_stk_1 * stk_1_daily_volatility) ** 2 \
                            + (weight_in_stk_3 * stk_3_daily_volatility) ** 2 \
                            + 2 * weight_in_stk_1 * weight_in_stk_3 * stk_1_3_cov[0, 1]

port_daily_volatility = np.sqrt(port_daily_variance)

print('The daily volatility of a portfolio with %s and %s is %.3f%%' % (stk_1, stk_3, port_daily_volatility))

In [ ]:
# Calculate the annualised volatility of the portfolio
port_annualized_volatility = port_daily_volatility * np.sqrt(252)

print('The annualised volatility of a portfolio with %s and %s is %.3f%%' % (stk_1, stk_3, port_annualized_volatility * 100))

In [ ]:
# Update HDFCBANK and MARICO data in 'comparison' dataframe
comparison.loc[stk_1 + '_' + stk_3, 'ann_returns'] = round(port_annualized_returns * 100, 3)
comparison.loc[stk_1 + '_' + stk_3, 'ann_volatility'] = round(port_annualized_volatility * 100, 3)

comparison.head()

In [ ]:
# Visualize the data
x = np.arange(4)
fig = plt.figure(figsize=(10, 6))
ax = fig.add_axes([0,0,1,1])
ax.bar(x + 0.00, comparison.ann_returns, color='teal', width = 0.25, label='Ann. Returns')
ax.bar(x + 0.25, comparison.ann_volatility, color='g', width = 0.25, label='Ann. Volatility')

ax.set_title('Comparison of Returns and Volatility')
ax.set_xticks(x)
ax.set_xticklabels(comparison.index)
ax.legend()

fig.tight_layout()
plt.show()

##### Questions to ask:

- Have we achieved our goal?

### e) Optimize Portfolio Weights

In [ ]:
np.arange(11, 20, step=2)

In [ ]:
# Create various portfolios with different weights
weight_in_stk_1 = np.arange(0, 1, step=0.01)

weight_in_stk_1

In [ ]:
weight_in_stk_3 = 1 - weight_in_stk_1

weight_in_stk_3

In [ ]:
# Create empty dataframe and dictionary to hold data for all portfolios
portfolios = pd.DataFrame(columns=[stk_1, stk_3, 'Ann_Returns', 'Ann_Volatility'])

# Create empty dictionary to hold new data for each portfolio
data_to_be_updated = {}

In [ ]:
# Iterate through each combination of weights in stock 1 and stock 3
for w_i in weight_in_stk_1:
    for w_d in weight_in_stk_3:
        
        # Select only those portfolios whose weights are equal to 1
        if (w_i + w_d) == 1:

            # Calculate expected returns for each portfolio
            port_daily_returns = (w_i * stk_1_data.daily_returns) + (w_d * stk_3_data.daily_returns)

            # Calculate cumulative returns for each portfolio
            port_cum_returns = compute_cum_returns(port_daily_returns)
            
            # Calculate annualized returns for each portfolio
            port_annualized_returns = compute_ann_returns(port_cum_returns[-1], len(port_cum_returns))

            # Calculate daily volatility for each portfolio
            port_daily_volatility = np.sqrt((w_i * stk_1_daily_volatility) ** 2 \
                                + (w_d * stk_3_daily_volatility) ** 2 \
                                + 2 * w_i * w_d * stk_1_3_cov[0, 1])

            # Calculate annualized volatility for each portfolio
            port_annualized_volatility = port_daily_volatility * np.sqrt(252)
            
            print(f'{stk_1}: {round(w_i, 2)} | {stk_3}: {round(w_d, 2)} | Ann. Returns: {round(port_annualized_returns, 4)} | Ann. Vol: {round(port_annualized_volatility, 4)}')
            
            # Update weights, annual returns and volatility the current portfolio
            data_to_be_updated[stk_1] = round(w_i, 2)
            data_to_be_updated[stk_3] = round(w_d, 2)
            data_to_be_updated['Ann_Returns'] = round(port_annualized_returns, 4)
            data_to_be_updated['Ann_Volatility'] = round(port_annualized_volatility, 4)
            
            # Append the current portfolio data to the dataframe
            portfolios = portfolios.append(data_to_be_updated, ignore_index=True)
else:
    print('Computations completed!')

In [ ]:
portfolios.head(20)

#### Minimum Risk Portfolio

In [ ]:
# Minimum Risk Portfolio
min_risk_portfolios = portfolios.loc[portfolios.Ann_Volatility == portfolios.Ann_Volatility.min()]

min_x = min_risk_portfolios.Ann_Volatility.iloc[0]
min_y = min_risk_portfolios.Ann_Returns.iloc[0]

min_risk_portfolios

#### Portfolio with Maximum Returns/Unit of Risk

In [ ]:
# Create a new column with maximum returns per unit of risk
portfolios['max_returns/risk'] = portfolios['Ann_Returns'] / portfolios['Ann_Volatility']

# Find the portfolio with maximum returns per unit of risk
sharpe_portfolios = portfolios.loc[portfolios['max_returns/risk'] == portfolios['max_returns/risk'].max()]

sharpe_x = sharpe_portfolios.Ann_Volatility.iloc[0]
sharpe_y = sharpe_portfolios.Ann_Returns.iloc[0]

sharpe_portfolios

In [ ]:
# Plotting it visually
fig = plt.figure(figsize=(15, 8))
plt.scatter(x='Ann_Volatility', y='Ann_Returns', data=portfolios, label='Various Portfolios')
plt.scatter(x=min_x, y=min_y, marker='*', s=200, color='darkorange', label='Minimum Risk Portfolio')
plt.scatter(x=sharpe_x, y=sharpe_y, marker='*', s=200, color='red', label='Max Returns/Risk Portfolio')
plt.xlabel('Annualized Volatility')
plt.ylabel('Annualized Returns')
plt.title('Portfolio Risk Vs Returns')
plt.legend(loc='best', fontsize=14)
plt.grid()
plt.show()

In [ ]:
import plotly.express as px
fig = px.scatter(portfolios, x="Ann_Volatility", y="Ann_Returns", hover_data=[stk_1, stk_3])
fig.show()

##### Questions to ask:

1. Did we lowered all the risk?
2. What can we do to further reduce the risk?
3. Can we reduce risk to 0?

---

# 4) Kelly Criterion for Binary Bets - For Self-Study

You have \\$100 and you bet **\\$40** on a bet. Following is the bet outcome:

- **Win Payout**: If you win, you get invested amount plus 10% of the invested amount.
    - Final amount would be 100 + (40 \* 0.1) = 104



- **Loss Payout**: If you lose, you lose the 30% of the invested amount.
    - Final amount would be 100 - (40 * 0.3) = 88
    

If you play this bet multiple times, *how much should you invest in each bet?*


In trading analogy,

- \\$100 = Initial Capital
- \\$40 = Fraction of capital invested
- Win Payout = Profit
- Loss Payout = Loss
- Bet = Trade

---

An investor begins with \\$1 and invests a fraction($k$) in an investment with two potential outcomes. 

- If the investment succeeds, it returns $B$ and the portfolio will be worth $1 + kB$. 
- If it fails, it loses $A$ and the portfolio will be worth $1 – kA$.

The profit after $n$ trades would be 

$$P_n = (1 + kB)^{wn} * (1 - kA)^{(1-w)n}$$

Where:

$ A = $ Loss payout *(Average loss per trade)*

$ B = $ Win payout *(Average profit per trade)*

$ n = $ Number of trades

$ w = $ Probability of winning *(# winning trades / total trades)*

$ 1 - w = $ Probability of losing *(# losing trades / total trades)*

-----
Rearranging the above formula when $n$ tends to infinite, we get:

$$ k = (w \div A) - (1-w \div B) $$

Where:

$ k = $ Fraction of wealth we want to invest

In [ ]:
initial_wealth = 1 

# These values are in percentage terms

# Define Payouts
loss_payout = 0.8 # (A | Average loss per trade) # 100% (Total loss for all loss making trades / # of loss making trades)
win_payout = 1.5 # (B | Average profit per trade) # 120 % (Total profit for all profitable trades / # of profitable trades)

# Define Probabilities (Sum of both probabilities should be equal to 1)
win_prob = 0.65 # (w | # winning trades / total trades)
loss_prob = 0.35 # 1 - win_prob (1-w | # losing trades / total trades)

# Total trades
n_trades = 500 # (n)

In [ ]:
# Calculate fraction of wealth to be traded using Kelly Criterion
k = (win_prob / loss_payout) - (loss_prob / win_payout)

print(f'One should invest {round(k, 2)} fraction of wealth in each trade.')

In [ ]:
# Calcuate wealth after 'n' trade
p = (1 + k * win_payout) ** (win_prob * n_trades) * (1 - k * loss_payout) ** (loss_prob * n_trades)

print(f'The final wealth would be ~{round(p, 3)}')

In [ ]:
# To Cross Verify
# Generate wealth for various fractions
fraction_list = np.arange(0, 1, 0.01)

# Create empty list to store wealth for each fraction
wealth = []

In [ ]:
fraction_list

In [ ]:
# Iterate over each fraction value and compute wealth for it
for k in fraction_list:
    
    # Calculate wealth
    p = (1 + k * win_payout)**(win_prob * n_trades) * (1 - k * loss_payout)**(loss_prob * n_trades)
    
    # Append the wealth value to the list
    wealth.append(p)
else:
    print('Computations completed!')

In [ ]:
# Plot wealth for all fractions
plt.figure(figsize=(10,6))
plt.plot(fraction_list, wealth)
plt.xlabel('Fraction of wealth')
plt.ylabel('Wealth')
plt.title('Kelly Criterion for Position Sizing')
plt.grid()
plt.show()

# 5) Application of Kelly Criterion

Read data file using which we will calculate the optimal `f` value using the Kelly. In other words, this works as the in-sample dataset.

In [ ]:
import yfinance as yf

In [ ]:
aapl_data_1518 = yf.download('aapl', start='2015-01-01', end='2018-12-31')

In [ ]:
# Define moving averages parameters
sma = 12
lma = 18

Define a function that backtests Moving Average Crossover strategy. We use this function to backtest the strategy on in-sample and out-of-sample data.

In [ ]:
def mvc_backtest(data, sma, lma, fraction):
    
    # Calculate daily percentage returns
    data['daily_returns'] = data['Adj Close'].pct_change()

    # Drop NaN values
    data.dropna(inplace=True)
    
    # Define moving averages - small and long, both
    data['sma'] = data['Adj Close'].rolling(sma).mean()
    data['lma'] = data['Adj Close'].rolling(lma).mean()
    
    # Define trading positions - long and short, both
    data['positions'] = np.where(data['sma'] > data['lma'], fraction, -fraction)
    
    # Calculate strategy returns
    data['s_returns'] = data['positions'].shift() * data['daily_returns']
    
    # Return the dataframe
    return data

Backtest the strategy on in-sample data using 100% wealth and store its results. That is, we invest all capital that we have in each trade.

In [ ]:
is_data = mvc_backtest(data=aapl_data_1518, sma=sma, lma=lma, fraction=1)

In [ ]:
cumulative_strategy_returns = (1 + is_data['s_returns']).cumprod()

print('Strategy returns:', round(cumulative_strategy_returns.iloc[-1] - 1, 2))

Calculate the optimal `f` value using the Kelly formula as shown below:

$$ f = \frac{\mu}{\sigma^2} $$

In [ ]:
# Calculate the mean returns
mean_returns = is_data['s_returns'].mean()

# Calculate the returns variance
variance_returns = is_data['s_returns'].var()

print('Mean:', round(mean_returns, 5), 'Variance:', round(variance_returns, 5))

# Calculate the optimal fraction to invest in each trade
optimal_fraction = mean_returns / variance_returns

print('Optimal fraction to invest in each trade:', round(optimal_fraction, 2))

Read the out-of-sample data and backtest it using all the wealth.

In [ ]:
aapl_data_1920 = yf.download('aapl', start='2019-01-01', end='2021-12-31')

Backtest the strategy using optimal fraction derived using Kelly on the in-sample data.

In [ ]:
oos_result_kelly = mvc_backtest(data=aapl_data_1920, sma=sma, lma=lma, fraction=optimal_fraction)

In [ ]:
cumulative_strategy_returns = (1 + oos_result_kelly['s_returns']).cumprod()

print('Strategy returns:', round(cumulative_strategy_returns.iloc[-1] - 1, 2))

Backtest the strategy using full capital for each trade.

In [ ]:
oos_result_100 = mvc_backtest(data=aapl_data_1920, sma=sma, lma=lma, fraction=1)

In [ ]:
cumulative_strategy_returns = (1 + oos_result_100['s_returns']).cumprod()

print('Strategy returns:', round(cumulative_strategy_returns.iloc[-1] - 1, 2))

# 5) Profitability Ratios

#### a) Sharpe Ratio

The Sharpe ratio is the excess return calculated as total returns less the risk-free rate of return per unit of volatility. Generally, risk-free return is the return on the risk-free assets such as government bonds. The excess returns are due to the 'extra risk' taken by the investor on investing in risky assets.

It tells whether the returns on a portfolio are due to good investment decision or the result of excessive risk taken. Higher Sharpe ratio is always preferable over the lower ones.

The Sharpe Ratio can be used to compare the portfolio with the benchmark to get to know how your portfolio is repaying for the risk taken on the investment.

$$ Sharpe\ Ratio\ =\ \frac{R_p - R_f}{\sigma_p} $$

Where:

$ R_p $ = Portfolio Returns

$ R_f $ = Risk-free Returns

$ \sigma_p $ = Standard deviation of the portfolio returns

In [ ]:
risk_free_rate = 0.00 # Annualized

# Calculate Sharpe Ratio
daily_sharpe = (np.mean(stk_1_data['daily_returns']) - (risk_free_rate/252))/ np.std(stk_1_data['daily_returns'])

# Calculate Annualized Sharpe Ratio
ann_sharpe = daily_sharpe * np.sqrt(252)

print("The Sharpe ratio is %.2f" % ann_sharpe)

In [ ]:
import empyrical as ep

In [ ]:
ep.sharpe_ratio(stk_1_data.daily_returns)

In [ ]:
ep.cagr(stk_1_data.daily_returns)

In [ ]:
comparison.head()

In [ ]:
# Daily returns of stock 1
pf.create_simple_tear_sheet(stk_1_data.daily_returns)

In [ ]:
# Tear sheet for Stock 1 and Stock 3
pf.create_simple_tear_sheet(stk_1_3_daily_returns)

#### b) Sortino Ratio
In the Sortino ratio, the denominator of the Sharpe ratio, the total standard deviation is replaced with the downside deviation. The downside deviation is the standard deviation of negative asset return.

It differentiates the harmful volatility from the total volatility by using the standard deviation of negative returns only. Since an investor is concerned only about the downside volatility, Sortino ratio is a good measure in comparing the highly volatile portfolios whereas the Sharpe ratio is better at analyzing portfolios with low volatility. The probability of large loss will be low if the value of the Sortino ratio is high.

$$ Sortino\ Ratio\ =\ \frac{R_p - R_f}{\sigma_d} $$

Where:

$ R_p $ = Portfolio Returns
<br>$ R_f $ = Risk-free Returns
<br>$ \sigma_d $ = Standard deviation of the negative asset returns

#### c) Treynor Ratio
Treynor Ratio is the variation in the denominator of the Sharpe ratio by replacing the total standard deviation with the beta of the portfolio. It also highlights the risk-adjusted performance of the portfolio. Higher the Treynor ratio, more suitable the investment is. The ratio is based on historical returns data, it is not necessary it will replicate in the future. The higher ratio tells that investment is good but it does not quantify how much good the investment is.

$$ Treynor\ Ratio\ =\ \frac{R_p - R_f}{\beta_p} $$

Where:

$ R_p $ = Portfolio Returns
<br>$ R_f $ = Risk-free Returns
<br>$ \beta_p $ = Portfolio's Beta

# Resources

- [Portfolio Management Of Multiple Strategies Using Python](https://blog.quantinsti.com/portfolio-management-strategy-python/)
- [Portfolio Optimization Methods](https://blog.quantinsti.com/portfolio-optimization-methods/)
- [Portfolio Analysis: Performance Measurement And Evaluation](https://blog.quantinsti.com/portfolio-analysis-performance-measurement-evaluation/)
- [Calculating The Covariance Matrix And Portfolio Variance](https://blog.quantinsti.com/calculating-covariance-matrix-portfolio-variance/)
- [Optimal Portfolio Construction Using Machine Learning](https://blog.quantinsti.com/optimal-portfolio-construction-machine-learning/)
- [Sharpe Ratio: Calculation, Application, Limitations](https://blog.quantinsti.com/sharpe-ratio-applications-algorithmic-trading/)
- [Volatility And Measures Of Risk-Adjusted Return With Python](https://blog.quantinsti.com/volatility-and-measures-of-risk-adjusted-return-based-on-volatility/)
---